In [9]:
import cv2
import numpy as np
import tensorflow as tf

# Загрузка модели TFLite
interpreter = tf.lite.Interpreter(model_path="models/volov8_model_v4.tflite")
interpreter.allocate_tensors()

# Получение информации о входных и выходных тензорах
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Выводим информацию о входной форме модели
print("Input details:", input_details)

# Функция предобработки изображения
def preprocess_image(image, input_shape):
    # Убедитесь, что изображение имеет 3 канала (RGB)
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    
    # Изменяем размер до (width, height)
    image = cv2.resize(image, (input_shape[3], input_shape[2]))
    image = image.astype(np.float32)
    image = image / 255.0  # нормализация
    image = np.expand_dims(image, axis=0)  # добавляем измерение batch
    image = np.transpose(image, (0, 3, 1, 2))  # меняем порядок размерностей на [batch, channels, height, width]
    return image

# Функция постобработки результатов (пример для YOLO)
def postprocess_output(output_data, img_width, img_height):
    boxes, scores, classes = [], [], []
    for detection in output_data[0]:
        score = detection[4]
        if score > 0.5:
            box = detection[:4] * np.array([img_width, img_height, img_width, img_height])
            boxes.append(box.astype(int))
            scores.append(score)
            classes.append(int(detection[5]))
    return boxes, scores, classes

# Инициализация камеры
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    input_shape = input_details[0]['shape']
    img_height, img_width, _ = frame.shape

    # Предобработка кадра
    input_data = preprocess_image(frame, input_shape)
    
    # Вывод отладочной информации о предобработанном изображении
    print("Preprocessed image shape:", input_data.shape)
    print("Preprocessed image dtype:", input_data.dtype)
    print("Expected input shape:", input_shape)

    # Установка данных в тензор
    try:
        interpreter.set_tensor(input_details[0]['index'], input_data)
    except ValueError as e:
        print(f"Error setting tensor: {e}")
        break

    # Выполнение инференса
    interpreter.invoke()

    # Получение результатов
    output_data = interpreter.get_tensor(output_details[0]['index'])
    boxes, scores, classes = postprocess_output(output_data, img_width, img_height)

    # Отображение результатов на кадре
    for box, score, cls in zip(boxes, scores, classes):
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        label = f"Class {cls}: {score:.2f}"
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Отображение кадра
    cv2.imshow('Object Detection', frame)

    # Выход по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Input details: [{'name': 'serving_default_images:0', 'index': 0, 'shape': array([  1,   3, 416, 416]), 'shape_signature': array([  1,   3, 416, 416]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Preprocessed image shape: (1, 3, 416, 416)
Preprocessed image dtype: float32
Expected input shape: [  1   3 416 416]
Preprocessed image shape: (1, 3, 416, 416)
Preprocessed image dtype: float32
Expected input shape: [  1   3 416 416]
Preprocessed image shape: (1, 3, 416, 416)
Preprocessed image dtype: float32
Expected input shape: [  1   3 416 416]
Preprocessed image shape: (1, 3, 416, 416)
Preprocessed image dtype: float32
Expected input shape: [  1   3 416 416]
Preprocessed image shape: (1, 3, 416, 416)
Preprocessed image dtype: float32
Expected input shape: [  1   3 416 416]


KeyboardInterrupt: 